## 1. Pobrac i wczytać do notebooka Ionosphere dataset

In [9]:
import pandas as pd

column_names = [f'attr{i}' for i in range(1, 35)] + ['class']

df = pd.read_csv('data/ionosphere.data', 
                     header=None,
                     names=column_names)

In [10]:
df

,attr1,attr2,attr3,attr4,attr5,attr6,attr7,attr8,attr9,attr10,...,attr26,attr27,attr28,attr29,attr30,attr31,attr32,attr33,attr34,class
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


## 2. Używając funkcji KFold z SciKit przygotować dane do sprawdzianu krzyżowego

In [ ]:
from sklearn.model_selection import KFold

X = df.drop('class', axis=1)
y = df['class']

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # Podział na zbiory treningowe i testowe dla danej iteracji
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print(f"Fold {fold+1}:")
    print(f"Train set size: {len(X_train)}, Test set size: {len(X_test)}")

Fold 1:
Train set size: 280, Test set size: 71
Fold 2:
Train set size: 281, Test set size: 70
Fold 3:
Train set size: 281, Test set size: 70
Fold 4:
Train set size: 281, Test set size: 70
Fold 5:
Train set size: 281, Test set size: 70


## 3. Wytrenować przy użyciu sprawdzianu krzyżowego drzewo decyzyjne, las losowy, regresję logistyczną, support vector machine i multi-layer perceptron, a następnie przeanalizować i porównać wyniki. Na jakie problemy trzeba zwrócić uwagę podczas treningu?

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def evaluate_classifier(classifier, X, y, train_test_splits):
    accuracies, precisions, recalls, f1_scores = [], [], [], []
    
    for fold, (train_index, test_index) in enumerate(train_test_splits):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, pos_label='g'))
        recalls.append(recall_score(y_test, y_pred, pos_label='g'))
        f1_scores.append(f1_score(y_test, y_pred, pos_label='g'))
        
        print(f"\nFold {fold+1}:")
        print(f"Accuracy: {accuracies[-1]:.4f}")
        print(f"Precision: {precisions[-1]:.4f}")
        print(f"Recall: {recalls[-1]:.4f}")
        print(f"F1-score: {f1_scores[-1]:.4f}")
    
    results = {
        'model': classifier.__class__.__name__,
        'accuracy_mean': np.mean(accuracies),
        'accuracy_std': np.std(accuracies),
        'precision_mean': np.mean(precisions),
        'precision_std': np.std(precisions),
        'recall_mean': np.mean(recalls),
        'recall_std': np.std(recalls),
        'f1_mean': np.mean(f1_scores),
        'f1_std': np.std(f1_scores)
    }
    
    print("\nŚrednie wyniki:")
    print(f"Accuracy: {results['accuracy_mean']:.4f} (±{results['accuracy_std']:.4f})")
    print(f"Precision: {results['precision_mean']:.4f} (±{results['precision_std']:.4f})")
    print(f"Recall: {results['recall_mean']:.4f} (±{results['recall_std']:.4f})")
    print(f"F1-score: {results['f1_mean']:.4f} (±{results['f1_std']:.4f})")
    
    return results

In [33]:
results = []

### 3.1. drzewo decyzyjne

In [34]:
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier(random_state=42)

results.append(evaluate_classifier(dt_classifier, X, y, kf.split(X)))


Fold 1:
Accuracy: 0.8873
Precision: 0.8571
Recall: 0.9767
F1-score: 0.9130

Fold 2:
Accuracy: 0.9000
Precision: 0.9318
Recall: 0.9111
F1-score: 0.9213

Fold 3:
Accuracy: 0.8714
Precision: 0.9348
Recall: 0.8776
F1-score: 0.9053

Fold 4:
Accuracy: 0.9143
Precision: 0.8864
Recall: 0.9750
F1-score: 0.9286

Fold 5:
Accuracy: 0.9000
Precision: 0.9184
Recall: 0.9375
F1-score: 0.9278

Średnie wyniki:
Accuracy: 0.8946 (±0.0144)
Precision: 0.9057 (±0.0297)
Recall: 0.9356 (±0.0380)
F1-score: 0.9192 (±0.0089)


### 3.2. las losowy


In [35]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

results.append(evaluate_classifier(rf_classifier, X, y, kf.split(X)))


Fold 1:
Accuracy: 0.9296
Precision: 0.9130
Recall: 0.9767
F1-score: 0.9438

Fold 2:
Accuracy: 0.9571
Precision: 0.9565
Recall: 0.9778
F1-score: 0.9670

Fold 3:
Accuracy: 0.9000
Precision: 0.9200
Recall: 0.9388
F1-score: 0.9293

Fold 4:
Accuracy: 0.9571
Precision: 0.9302
Recall: 1.0000
F1-score: 0.9639

Fold 5:
Accuracy: 0.9000
Precision: 0.9556
Recall: 0.8958
F1-score: 0.9247

Średnie wyniki:
Accuracy: 0.9288 (±0.0256)
Precision: 0.9351 (±0.0180)
Recall: 0.9578 (±0.0367)
F1-score: 0.9457 (±0.0173)


### 3.3. regresja logistyczna

In [36]:
from sklearn.linear_model import LogisticRegression

lr_classifier = LogisticRegression(
    random_state=42,
    max_iter=1000
)

results.append(evaluate_classifier(lr_classifier, X, y, kf.split(X)))


Fold 1:
Accuracy: 0.8592
Precision: 0.8113
Recall: 1.0000
F1-score: 0.8958

Fold 2:
Accuracy: 0.8857
Precision: 0.8776
Recall: 0.9556
F1-score: 0.9149

Fold 3:
Accuracy: 0.8429
Precision: 0.8519
Recall: 0.9388
F1-score: 0.8932

Fold 4:
Accuracy: 0.8857
Precision: 0.8333
Recall: 1.0000
F1-score: 0.9091

Fold 5:
Accuracy: 0.9143
Precision: 0.9200
Recall: 0.9583
F1-score: 0.9388

Średnie wyniki:
Accuracy: 0.8775 (±0.0246)
Precision: 0.8588 (±0.0375)
Recall: 0.9705 (±0.0250)
F1-score: 0.9104 (±0.0163)


### 3.4. support vector machines

In [37]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', random_state=42)

results.append(evaluate_classifier(svm_classifier, X, y, kf.split(X)))


Fold 1:
Accuracy: 0.8732
Precision: 0.8269
Recall: 1.0000
F1-score: 0.9053

Fold 2:
Accuracy: 0.9000
Precision: 0.8958
Recall: 0.9556
F1-score: 0.9247

Fold 3:
Accuracy: 0.8000
Precision: 0.8182
Recall: 0.9184
F1-score: 0.8654

Fold 4:
Accuracy: 0.8571
Precision: 0.8125
Recall: 0.9750
F1-score: 0.8864

Fold 5:
Accuracy: 0.8857
Precision: 0.8846
Recall: 0.9583
F1-score: 0.9200

Średnie wyniki:
Accuracy: 0.8632 (±0.0346)
Precision: 0.8476 (±0.0353)
Recall: 0.9615 (±0.0267)
F1-score: 0.9003 (±0.0220)


### 3.5. mulit-layer perceptron

In [38]:
from sklearn.neural_network import MLPClassifier

mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

results.append(evaluate_classifier(mlp_classifier, X, y, kf.split(X)))


Fold 1:
Accuracy: 0.9296
Precision: 0.8958
Recall: 1.0000
F1-score: 0.9451

Fold 2:
Accuracy: 0.9571
Precision: 0.9565
Recall: 0.9778
F1-score: 0.9670

Fold 3:
Accuracy: 0.9286
Precision: 0.9231
Recall: 0.9796
F1-score: 0.9505

Fold 4:
Accuracy: 0.9143
Precision: 0.8696
Recall: 1.0000
F1-score: 0.9302

Fold 5:
Accuracy: 0.9000
Precision: 0.9184
Recall: 0.9375
F1-score: 0.9278

Średnie wyniki:
Accuracy: 0.9259 (±0.0190)
Precision: 0.9127 (±0.0290)
Recall: 0.9790 (±0.0228)
F1-score: 0.9441 (±0.0143)


### Podsumowanie wyników

In [40]:
comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.set_index('model')
comparison_df = comparison_df.round(4)

print("\nPorównanie wszystkich modeli:")
comparison_df


Porównanie wszystkich modeli:


,accuracy_mean,accuracy_std,precision_mean,precision_std,recall_mean,recall_std,f1_mean,f1_std
model,,,,,,,,
DecisionTreeClassifier,0.8946,0.0144,0.9057,0.0297,0.9356,0.0380,0.9192,0.0089
RandomForestClassifier,0.9288,0.0256,0.9351,0.0180,0.9578,0.0367,0.9457,0.0173
LogisticRegression,0.8775,0.0246,0.8588,0.0375,0.9705,0.0250,0.9104,0.0163
SVC,0.8632,0.0346,0.8476,0.0353,0.9615,0.0267,0.9003,0.0220
MLPClassifier,0.9259,0.0190,0.9127,0.0290,0.9790,0.0228,0.9441,0.0143


 ### Na jakie problemy trzeba zwrócić uwagę podczas treningu?

1. Dobór hiperparametrów
2. Przeuczenie (overfitting) modelu
3. Skalowanie danych (szczególnie w kontekście SVM, MLP) - dane nie były skalowane, więc może dlatego te klasyfikatory wypadają gorzej od pozostałych
4. Niezbalansowanie klas
5. Złożoność obliczeniowa (niektóre modele potrzebują więcej czasu podczas treningu)

## 4. Przygotować się do dyskusji. Jaka metoda wypadła najlepiej? Czy rezultat był oczekiwany? Jeżeli tak to dlaczego, jeżeli nie, to jaka metoda była spodziewana, żeby dać najlepsze wyniki? 

### 4.1. Jaka metoda wypadła najlepiej?

1. miejsce: Random Forest Classifier:
- Najwyższa dokładność (accuracy): 0.9288 (±0.0256)
- Najwyższa precyzja (precision): 0.9351 (±0.0180)
- Najwyższy średni F1-score: 0.9457 (±0.0173)

2. miejsce: MLP (Multi-Layer Perceptron):
- Wysoka dokładność: 0.9259 (±0.0190)
- Najwyższy recall: 0.9790 (±0.0228)
- Wysoki F1-score: 0.9441 (±0.0143)


### 4.2. Czy rezultat był oczekiwany?

Tak, ponieważ RF radzi sobie dobrze, gdy:
- Mamy stosunkowo małą liczbę próbek (351)
- Dane są numeryczne
- RF dobrze radzi sobie z nieliniowymi zależnościami w danych
- Jest odporny na przeuczenie dzięki agregacji wielu drzew